# Process the data
This notebook shows a simple example of extracting data at a point on the downloaded xarray from the `load_data.ipynb` file. At the end of that file, I ran `%store ds` where `ds` is the xarray of SST values.

This compares 2 ways of getting SST from my data points (track). The first is very slow. The second is much faster, so scroll to the bottom.

In [1]:
# This is stored from load_data.ipynb file so run that first
%store -r ds

# Get SST at a particular lat/lon value
I have my lat/lon data in a file `data/sample_point_pairs.csv`. These are points along the North and South America coasts. My lat/lon values are not on the same grid as the SST data, so I need to find the "pixel" where my lat/lon value lies. 

In [2]:
import pandas as pd
df = pd.read_csv('../data/sample_point_pairs.csv')

Here is a lat/lon pair.

In [3]:
ns = [df.iloc[0]["lat.ns"], df.iloc[0]["lon.ns"]]
ns

[54.8932363035493, -164.839138084371]

Here is the code to get SST (in Kelvin) at a particular lat/lon point.

In [4]:
ds.analysed_sst.sel(lat=ns[0], lon=ns[1], method="nearest").values[0]

278.32

# Alternate approach using indices
From https://github.com/ICESAT-2HackWeek/xarray_open_tutorial/blob/master/xarray_examples.ipynb

In [5]:
import xarray as xr
import numpy as np

In [6]:
ind_x = xr.DataArray(df["lon.ns"], dims=['i'])
ind_y = xr.DataArray(df["lat.ns"], dims=['i'])

In [7]:
test = ds.analysed_sst.sel(lon=ind_x, lat=ind_y, method='nearest').values
test[0, 0:20]

array([278.32   , 278.00998, 277.69998, 278.00998, 278.03   , 277.83   ,
       277.29   , 274.81   , 273.84   , 271.77   , 272.74   , 272.18   ,
       271.94998, 271.85   , 271.66998, 271.61   , 271.77   , 271.69   ,
       271.6    ,       nan], dtype=float32)

Make a function to do this.

In [8]:
def getdf2(ras, pts):
    ind_x = xr.DataArray(df["lon.ns"], dims=['i'])
    ind_y = xr.DataArray(df["lat.ns"], dims=['i'])
    xr_new = ras.analysed_sst.sel(lon=ind_x, lat=ind_y, method='nearest').values
    ind_x = xr.DataArray(df["lon.os"], dims=['i'])
    ind_y = xr.DataArray(df["lat.os"], dims=['i'])
    xr_new = np.vstack((xr_new, ras.analysed_sst.sel(lon=ind_x, lat=ind_y, method='nearest').values))
    return xr_new

In [9]:
%timeit getdf2(ds, df)

157 ms ± 4.62 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


This is much faster!

101.91082802547771

In [24]:
ind_x = xr.DataArray([[df["lon.ns"][1]-0.25,df["lon.ns"][1]]], dims=['i','y'])
ind_y = xr.DataArray([[df["lat.ns"][1]-0.25,df["lat.ns"][1]]], dims=['i','y'])
test = ds.analysed_sst.sel(lon=ind_x, lat=ind_y, method='nearest').values
test

array([[[278.00998, 278.00998]]], dtype=float32)

In [25]:
test = ds.analysed_sst.sel(lon=ind_x, lat=ind_y, method='nearest').values
test

array([[[278.25998,       nan]]], dtype=float32)

In [26]:
ind_x

<xarray.DataArray (i: 1, y: 2)>
array([[-164.35862178, -164.35862178]])
Dimensions without coordinates: i, y

In [33]:
import rioxarray
import rasterio
sst_da = ds.analysed_sst.rio.write_crs("+proj=longlat +datum=WGS84 +no_defs")
wintri_crs = '+proj=wintri +lon_0=0 +lat_1=0 +x_0=0 +y_0=0 +datum=WGS84 +units=km +no_defs'
sst_da_reproj = sst_da.rio.reproject(wintri_crs)

Step one is to add a projection to our xarray since it doesn't have one. In order to do a re-projection, we have to know what we are re-projecting from. The projection of the SST data in PROJ4 format is "+proj=longlat +datum=WGS84 +no_defs". Lots of ways to refer to this projection. "epsg:4326" is also common. I use PROJ4 normally.

In [34]:
sst_da_reproj.shape

(1, 1138, 2277)

In [48]:
sst_da_reproj.y.values[1:10]-sst_da_reproj.y.values[0:9]


array([-17.60115694, -17.60115694, -17.60115694, -17.60115694,
       -17.60115694, -17.60115694, -17.60115694, -17.60115694,
       -17.60115694])